In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}

In [2]:
-- import Prelude hiding ((++))
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA

import qualified Data.Map as M
import Data.Map ((!))

import Control.Monad (when, unless)
import Control.Monad.State.Lazy
import Control.Monad.Reader
import Control.Monad.Writer

In [3]:
type TuringState = String

type Tape = M.Map Integer Bool

data StateTransition = StateTransition { writeValue :: Bool
                                       , newState :: TuringState
                                       , tapeMovement :: Integer
                                       } deriving (Show, Eq)

type RuleTrigger = (TuringState, Bool)

type Rules = M.Map RuleTrigger StateTransition

data Machine = Machine { state :: TuringState
                       , tape :: Tape
                       , tapeLocation :: Integer
                       , stepsRemaining :: Integer
                       } 
               deriving (Show, Eq)

emptyMachine = Machine {state = "unknown", tape = M.empty, tapeLocation = 0, stepsRemaining = 0}

type ProgrammedMachine = ReaderT Rules (State Machine) Int

In [4]:
sc :: Parser ()
sc = L.space (skipSome spaceChar) CA.empty CA.empty

lexeme = L.lexeme sc
integer = lexeme L.integer
symbol = L.symbol sc
fullstop = symbol "."

commandP = between (symbol "-") fullstop

writeValueP = (symbol "1" *> pure True) <|> (symbol "0" *> pure False)
writeP = commandP ((symbol "Write the value") *> writeValueP)

directionP = (symbol "left" *> pure -1) <|> (symbol "right" *> pure 1)
tapeMovementP = commandP ((symbol "Move one slot to the") *> directionP)

newStateP = commandP ((symbol "Continue with state") *> (some letterChar))

stateTransitionP = stify <$> writeP <*> tapeMovementP <*> newStateP
    where stify w t s = StateTransition {writeValue = w, newState = s, tapeMovement = t}
    
currentValueP = (symbol "If the current value is") *> writeValueP <* (symbol ":")
    
stateWhenP = (,) <$> currentValueP <*> stateTransitionP
    
stateDefP = (symbol "In state") *> (some letterChar) <* (symbol ":")
    
stateRulesP = rulify <$> stateDefP <*> (stateWhenP `sepBy` space)
    where rulify s ts = M.fromList $ map (\(v, t) -> ((s, v), t)) ts
   
manyStateRulesP = M.unions <$> (stateRulesP `sepBy` space)

startStateP = (symbol "Begin in state") *> (some letterChar) <* fullstop
stepsP =  (symbol "Perform a diagnostic checksum after") *> integer <* (symbol "steps") <* fullstop

machineDescriptionP = machineify <$> startStateP <*> stepsP <*> manyStateRulesP
    where machineify initial limit rules = 
            ( emptyMachine { state = initial, stepsRemaining = limit }
            , rules
            )

In [5]:
parseTest writeP "- Write the value 1."

True

In [6]:
parseTest tapeMovementP "- Move one slot to the right."

1

In [7]:
parseTest newStateP "- Continue with state Fallow."

"Fallow"

In [8]:
parseTest stateTransitionP "- Write the value 1.\n    - Move one slot to the right.\n    - Continue with state B."

StateTransition {writeValue = True, newState = "B", tapeMovement = 1}

In [9]:
parseTest stateWhenP "If the current value is 1:\n    - Write the value 1.\n    - Move one slot to the right.\n    - Continue with state B."

(True,StateTransition {writeValue = True, newState = "B", tapeMovement = 1})

In [10]:
parseTest stateDefP "In state A:"

"A"

In [11]:
parseTest (stateWhenP `sepBy` space) "If the current value is 0:\n    - Write the value 1.\n    - Move one slot to the right.\n - Continue with state B.\n If the current value is 1:\n - Write the value 0.\n - Move one slot to the left.\n - Continue with state F.\n"

[(False,StateTransition {writeValue = True, newState = "B", tapeMovement = 1}),(True,StateTransition {writeValue = False, newState = "F", tapeMovement = -1})]

In [12]:
parseTest stateRulesP "In state A:\nIf the current value is 0:\n    - Write the value 1.\n    - Move one slot to the right.\n - Continue with state B.\n If the current value is 1:\n - Write the value 0.\n - Move one slot to the left.\n - Continue with state F.\n"

fromList [(("A",False),StateTransition {writeValue = True, newState = "B", tapeMovement = 1}),(("A",True),StateTransition {writeValue = False, newState = "F", tapeMovement = -1})]

In [13]:
parseTest startStateP "Begin in state A."

"A"

In [14]:
parseTest stepsP "Perform a diagnostic checksum after 12994925 steps."

12994925

In [15]:
successfulParse :: Text -> (Machine, Rules)
successfulParse input = 
        case parse machineDescriptionP "input" input of
                Left  _error -> (emptyMachine, M.empty)
                Right machineRules -> machineRules

In [16]:
text <- TIO.readFile "../../data/advent25.txt"
successfulParse text

(Machine {state = "A", tape = fromList [], tapeLocation = 0, stepsRemaining = 12994925},fromList [(("A",False),StateTransition {writeValue = True, newState = "B", tapeMovement = 1}),(("A",True),StateTransition {writeValue = False, newState = "F", tapeMovement = -1}),(("B",False),StateTransition {writeValue = False, newState = "C", tapeMovement = 1}),(("B",True),StateTransition {writeValue = False, newState = "D", tapeMovement = 1}),(("C",False),StateTransition {writeValue = True, newState = "D", tapeMovement = -1}),(("C",True),StateTransition {writeValue = True, newState = "E", tapeMovement = 1}),(("D",False),StateTransition {writeValue = False, newState = "E", tapeMovement = -1}),(("D",True),StateTransition {writeValue = False, newState = "D", tapeMovement = -1}),(("E",False),StateTransition {writeValue = False, newState = "A", tapeMovement = 1}),(("E",True),StateTransition {writeValue = True, newState = "C", tapeMovement = 1}),(("F",False),StateTransition {writeValue = True, newState

In [24]:
text <- TIO.readFile "advent25sample.txt"
(sampleMachine, sampleRules) = successfulParse text
(sampleMachine, sampleRules)

(Machine {state = "A", tape = fromList [], tapeLocation = 0, stepsRemaining = 6},fromList [(("A",False),StateTransition {writeValue = True, newState = "B", tapeMovement = 1}),(("A",True),StateTransition {writeValue = False, newState = "B", tapeMovement = -1}),(("B",False),StateTransition {writeValue = True, newState = "A", tapeMovement = -1}),(("B",True),StateTransition {writeValue = True, newState = "A", tapeMovement = 1})])

In [26]:
executeStep = 
    do rules <- ask
       m <- get
       let tapeHere = M.findWithDefault False (tapeLocation m) (tape m)
       let transition = rules!(state m, tapeHere)
       let tape' = M.insert (tapeLocation m) (writeValue transition) (tape m)
       let loc' = (tapeLocation m) + (tapeMovement transition)
       let state' = newState transition
       let steps' = stepsRemaining m - 1
       let m' = m {state = state', tape = tape', tapeLocation = loc', stepsRemaining = steps'}
       put m'
       

Line 7: Redundant bracket
Found:
(tapeLocation m) + (tapeMovement transition)
Why not:
tapeLocation m + (tapeMovement transition)Line 7: Redundant bracket
Found:
(tapeLocation m) + (tapeMovement transition)
Why not:
(tapeLocation m) + tapeMovement transition

In [27]:
executeSteps = 
    do m <- get
       unless (stepsRemaining m == 0) $
           do  executeStep
               executeSteps

In [28]:
part1 rules machine0 = 
    runState (
        runReaderT executeSteps
                   rules 
             ) 
             machine0

Line 1: Eta reduce
Found:
part1 rules machine0
  = runState (runReaderT executeSteps rules) machine0
Why not:
part1 rules = runState (runReaderT executeSteps rules)

In [35]:
part1 rules machine0 = 
    execState (
        runReaderT executeSteps
                   rules 
             ) 
             machine0

Line 1: Eta reduce
Found:
part1 rules machine0
  = execState (runReaderT executeSteps rules) machine0
Why not:
part1 rules = execState (runReaderT executeSteps rules)

In [36]:
execState ( runReaderT executeStep sampleRules ) sampleMachine

Machine {state = "B", tape = fromList [(0,True)], tapeLocation = 1, stepsRemaining = 5}

In [37]:
main :: IO ()
main = do 
        text <- TIO.readFile "../../data/advent25.txt"
        let (machine0, rules) = successfulParse text
        let machinef = part1 rules machine0
        print $ M.size $ M.filter id $ tape machinef


In [38]:
-- main :: IO ()
-- main = do 
--         text <- TIO.readFile "advent25sample.txt"
--         let (machine0, rules) = successfulParse text
--         let (result, machinef) = part1 rules machine0
--         print $ M.size $ M.filter id $ tape machinef


In [39]:
main

2846